# Info 558 Spring 2022 - Kaggle Notebook - Joshin Kumar

* ### Code written for ensembling predictions from models available on Keras.
* ### Open source code for generating the TFRecord files was edited.

The TPU code is based off of Martin Gorner's excellent starter notebook, and is intended solely as a simple, shorter introduction to the operations being performed there.

In [ ]:
import tensorflow as tf
from kaggle_datasets import KaggleDatasets
import numpy as np

print("Tensorflow version " + tf.__version__)

# Detect my accelerator

In [ ]:
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

# Get my data path

In [ ]:
GCS_DS_PATH = KaggleDatasets().get_gcs_path() # you can list the bucket with "!gsutil ls $GCS_DS_PATH"
display(GCS_DS_PATH)

!gsutil ls $GCS_DS_PATH

# Set some parameters

In [ ]:
IMAGE_SIZE = [512, 512] #FIXED@768 For Best Results // at this size, a GPU will run out of memory. Use the TPU #

# EfficientNetB3 Time_per_epoch_found from Trial Run = 1min/epoch
# REMINDER ##############FOR EFFECIENTNET 0-255 Scale Check?????????????
EPOCHS = 1
PATIENCE = 1
BATCH_SIZE = 8 * strategy.num_replicas_in_sync

NUM_TRAINING_IMAGES = 16018#
NUM_TEST_IMAGES = 4005#
STEP_SIZE_TRAIN = NUM_TRAINING_IMAGES // BATCH_SIZE
STEP_SIZE_VALID = NUM_TEST_IMAGES // BATCH_SIZE
AUTO = tf.data.experimental.AUTOTUNE

display([*IMAGE_SIZE, 3])

# Load my data

This data is loaded from Kaggle and automatically sharded to maximize parallelization.

In [ ]:
def decode_image(image_data):
    image = tf.image.decode_jpeg(image_data, channels=3)
    image = tf.cast(image, tf.float32) / 255.0  # convert image to floats in [0, 1] range // Comment 255 only for EfficientNetB6
    image = tf.image.resize(image, [*IMAGE_SIZE])
    
#     display("Here is the image: ", image)
#     display(f"Point1: Image Shape = {image.shape} OR {tf.shape(image)} ||  and Image Length: {tf.size(image)} OR {len(image)}")
#     image = tf.reshape(image, [*IMAGE_SIZE, 3]) # explicit size needed for TPU
#     display(f"Point2: Image Shape = {image.shape} OR {tf.shape(image)} ||  and Image Length: {tf.size(image)}")

    return image

def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "filename": tf.io.FixedLenFeature([], tf.int64),
        "distance": tf.io.FixedLenFeature([], tf.int64),  # shape [] means single element
    }
    example = tf.io.parse_single_example(example, LABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    label = tf.cast(example['distance'], tf.int32)
    return image, label # returns a dataset of (image, label) pairs

def read_unlabeled_tfrecord(example):
    UNLABELED_TFREC_FORMAT = {
        "image": tf.io.FixedLenFeature([], tf.string), # tf.string means bytestring
        "image_name": tf.io.FixedLenFeature([], tf.string),
        "filename": tf.io.FixedLenFeature([], tf.int64),      
#         "id": tf.io.FixedLenFeature([], tf.string),  # shape [] means single element
        # class is missing, this competitions's challenge is to predict flower classes for the test dataset
    }
    example = tf.io.parse_single_example(example, UNLABELED_TFREC_FORMAT)
    image = decode_image(example['image'])
    idnum = example['image_name']
    return image, idnum # returns a dataset of image(s)

def load_dataset(filenames, labeled=True, ordered=False):
    # Read from TFRecords. For optimal performance, reading from multiple files at once and
    # disregarding data order. Order does not matter since we will be shuffling the data anyway.

    ignore_order = tf.data.Options()
    if not ordered:
        ignore_order.experimental_deterministic = False # disable order, increase speed

    dataset = tf.data.TFRecordDataset(filenames, num_parallel_reads=AUTO) # automatically interleaves reads from multiple files
    dataset = dataset.with_options(ignore_order) # uses data as soon as it streams in, rather than in its original order
    dataset = dataset.map(read_labeled_tfrecord if labeled else read_unlabeled_tfrecord, num_parallel_calls=AUTO)
    # returns a dataset of (image, label) pairs if labeled=True or (image, id) pairs if labeled=False
    return dataset

def get_training_dataset():
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/TFRecords_Files/train*.tfrec'), labeled=True)
    dataset = dataset.repeat() # the training dataset must repeat for several epochs
    dataset = dataset.shuffle(2048)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_validation_dataset():
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/TFRecords_Files/train*.tfrec'), labeled=True, ordered=False) # For now: Training == Validation/ Need to upload validation tfrecord files
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.cache()
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

def get_test_dataset(ordered=False):
    dataset = load_dataset(tf.io.gfile.glob(GCS_DS_PATH + '/TFRecords_Files/test*.tfrec'), labeled=False, ordered=ordered)
    dataset = dataset.batch(BATCH_SIZE)
    dataset = dataset.prefetch(AUTO) # prefetch next batch while training (autotune prefetch buffer size)
    return dataset

training_dataset = get_training_dataset()
validation_dataset = get_validation_dataset()

In [ ]:
display(training_dataset)

In [ ]:
# !pip install tf-nightly

# Build a model on TPU (or GPU, or CPU...) with Tensorflow 2.1!

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input

from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Flatten
from tensorflow.keras.models import Model

from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError

###### Define Model ######
with strategy.scope():
    input_tensor = Input(shape=([*IMAGE_SIZE, 3]))

    base_model = ResNet50(include_top=None, weights=None, input_tensor=input_tensor, input_shape=None)
    # Used different base_models ResNets, EfficientNets and Xceptions and finally weighted averaged the predictions to get the final prediction.

    x=base_model.output
    x=GlobalAveragePooling2D()(x)
    x=Flatten()(x)
    x=Dense(1024,activation='relu')(x) # 2 Layers of 1024 Neurons Origionally // Change this for different base_models for best results
    x=Dense(1024,activation='relu')(x) 

    model=Model(inputs=base_model.input,outputs=Dense(1)(x))
    
    metric_used = RootMeanSquaredError(name="rmse")

    

model.summary()

In [ ]:
###### Compile and Fit ######
display(f"Model Parameters # IMAGE_SIZE={IMAGE_SIZE} # BATCH_SIZE={BATCH_SIZE} # EPOCHS={EPOCHS} # PATIENCE={PATIENCE} # STEP_SIZE_TRAIN={STEP_SIZE_TRAIN} # STEP_SIZE_VALID={STEP_SIZE_VALID} #")

model.compile(loss = 'mean_squared_error', optimizer='adam', metrics=metric_used) #steps_per_execution =>[DOES NOT WORK! IF ERROR => CHANGE BATCH_SIZE INSTEAD]

monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=PATIENCE, verbose=1, mode='auto', restore_best_weights=True)

history = model.fit(training_dataset, epochs=EPOCHS, validation_data = validation_dataset, callbacks=[monitor],
                    verbose = 1, steps_per_epoch=STEP_SIZE_TRAIN, validation_steps=STEP_SIZE_VALID)


# Compute your predictions on the test set!

This will create a file that can be submitted to the competition.

In [ ]:
test_ds = get_test_dataset(ordered=True) # since we are splitting the dataset and iterating separately on images and ids, order matters.

print('Computing predictions...')
test_images_ds = test_ds.map(lambda image, idnum: image)
probabilities = model.predict(test_images_ds)
predictions = probabilities # Regression Output
# predictions = np.argmax(probabilities, axis=-1)
print(predictions)


print('Generating submission.csv file...')
test_ids_ds = test_ds.map(lambda image, idnum: idnum).unbatch()
test_ids = next(iter(test_ids_ds.batch(NUM_TEST_IMAGES))).numpy().astype('U') # all in one batch

predictions = predictions.flatten()
display(test_ids.shape)
display(predictions.shape)


np.savetxt(f'Submission-ResNet50-R512.csv', np.rec.fromarrays([test_ids, predictions]), fmt=['%s', '%d'], delimiter=',', header='id,distance', comments='')


In [ ]:
# Saving the Trained Model
# model.save('ResNet50-TPU.h5')

#Loading the saved Model
# model = tf.keras.models.load_model('../input/yourDataset/model.h5') # This saved model can be used to further train, if results are improving.



In [ ]:
# ################################ ENSEMBLE CODE ################################

# import numpy as np
# import pandas as pd
# import os

# PATH = r"C:\Users\joshi\Desktop\Kaggle Competition"
# P1 = os.path.join(PATH, "Submission-ResNet50-Final.csv")
# P2 = os.path.join(PATH, "Ensemble-Remix-2-Final.csv")
# P3 = os.path.join(PATH, "Ensemble-ALL-Remix-Final.csv")
# # P4 = os.path.join(PATH, "Submission-EfficientNetB2-R512.csv")
# # P5 = os.path.join(PATH, "Submission-EfficientNetB3-R512.csv")
# # P6 = os.path.join(PATH, "Submission-EfficientNetB7-R512.csv")


# df1 = pd.read_csv(P1)
# df2 = pd.read_csv(P2)
# df3 = pd.read_csv(P3)
# # df4 = pd.read_csv(P4)
# # df5 = pd.read_csv(P5)
# # df6 = pd.read_csv(P5)


# # display(df2)

# df_Ensemble = df1
# df_Ensemble["distance"] = (0.5*df1["distance"] + 0.5*df2["distance"] + 2*df3["distance"])/3 #+ df4["distance"] + df5["distance"] + df6["distance"])/6

# # display(df1.head(),
# # df2.head(),
# # df_Ensemble.head())


# df_Ensemble.to_csv(r"C:\Users\joshi\Desktop\Kaggle Competition\Ensemble-Remix-Final.csv", index=False)